In [41]:
import pinecone

In [42]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [43]:
PINECONE_API_KEY = "316c44ff-646d-4332-8db6-5ea2d25ce8dc"
PINECONE_API_ENV ="us-east-1"

In [44]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [45]:
extracted_data = load_pdf("E:\question_bot\data/")

Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x247 for key /Subtype


Multiple definitions in dictionary at byte 0x8acf for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x342 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x342 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x342 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype


In [46]:
extracted_data

[Document(metadata={'source': 'E:\\question_bot\\data\\Chatbot_in_business.pdf', 'page': 0}, page_content=' \n \nThis is an open access paper under the license  \n \n I C E M I T  \n1st International Scientific Conference on Economy, Management \nand Informa tion Technologies – ICEMIT 2023  \n \n \nChatbot in business  \n \nMilan Puvačaa\uf02a, Ivan Kunacb \n \na Ofir d.o.o., Osijek, Croatia  \nb A. G. Matoš Elementary School, Tovarnik, Croatia  \n \nA r t i c l e   i n f o   A b s t r a c t  \n \nReview paper  \n \nDOI:  \nhttps://doi.org/10.46793/ICEMIT23.219P       \n \nUDC/ UDK:  \n070.3 :004.8    \nThe cost of today\'s employee education is ex tensive, but also dynamic due to the global \nchallenges of business that encompasses each activity. The special expression of the \naforementioned is outlined in the elements of customer support provided to visitors or \npotential clients through the websites o f companies or other institutions. In order to \nsomewhat bridge the gap of new 

In [47]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [48]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 83


In [49]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [50]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'


In [51]:
embeddings = download_hugging_face_embeddings()

In [52]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [53]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [54]:
import pinecone
from pinecone import Pinecone,ServerlessSpec

In [55]:
from pinecone import Pinecone

# Initialize Pinecone with your API key and environment
pinecone = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)


In [56]:
pinecone

In [57]:
index_name = "question-bot"
# Your other operations with Pinecone


In [60]:

#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="question-bot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [59]:
import pinecone

# Initialize Pinecone with environment and API key
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# Create a client instance
client = pinecone.Client()

# Load the existing index
index_name = "question-bot"
index = client.Index(index_name)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [1]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are business"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

NameError: name 'Pinecone' is not defined

In [4]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [5]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

NameError: name 'PromptTemplate' is not defined

In [9]:
!pip install ctransformers


In [13]:
import ctransformers
print(dir(ctransformers))
!pip install --upgrade ctransformers


['AutoConfig', 'AutoModelForCausalLM', 'AutoTokenizer', 'Config', 'LLM', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'hub', 'lib', 'llm', 'logger', 'utils']


In [14]:
from ctransformers import CTransformers

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)


ImportError: cannot import name 'CTransformers' from 'ctransformers' (c:\Users\realme\anaconda3\envs\vivek2110\lib\site-packages\ctransformers\__init__.py)

In [6]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

NameError: name 'CTransformers' is not defined

In [7]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'RetrievalQA' is not defined